### Github Notebook Repo: https://github.com/ppalmes

# [DFA Preliminaries](./DFA.pdf)

In [1]:
nprocs()==1 && addprocs()

@everywhere logis(x) = 1 ./ (1 .+ exp(-x))
@everywhere dlogis(x) = logis(x) .* (1 .- logis(x))

# @everywhere logis(x) = sign(x)
# @everywhere dlogis(x) = 0


@everywhere function initParams(x,y,n_hidden,max_iter)
    w1 = randn(n_hidden,size(x,1))
    w2 = randn(n_hidden,n_hidden)
    w3 = randn(size(y,1),n_hidden)  
    B1 = rand(n_hidden,1)
    B2 = rand(n_hidden,1)      
    errors = zeros(max_iter)
    outputs = zeros(size(y,2),max_iter)
    return (w1,w2,w3,errors,outputs,B1,B2)
end

@everywhere function ffward(x,y,w1,w2,w3)
    a1 = w1 * x
    h1 = logis(a1)
    a2 = w2 * h1
    h2 = logis(a2)
    ay = w3 * h2
    yhat = logis(ay)
    err = yhat - y
    return(a1,h1,a2,h2,ay,yhat,err)
end

@everywhere function updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err;lr=0.2)
    w1 .-= δa1 * x' .* lr
    w2 .-= δa2 * h1' .* lr
    w3 .-= err * h2' .* lr
end

@everywhere function bp(x,y,n_hidden,max_iter)
    (w1,w2,w3,errors,outputs,_,_)=initParams(x,y,n_hidden,max_iter)
    for i = 1:max_iter   
        (a1,h1,a2,h2,ay,yhat,err) = ffward(x,y,w1,w2,w3)
        δa2 = (w3' * err) .* dlogis(a2)
        δa1 = (w2' * δa2) .* dlogis(a1)
        updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err)
        errors[i] = sum(abs(err))
        outputs[:,i] = yhat
    end
    return (outputs,errors)
end

@everywhere function fa(x,y,n_hidden,max_iter)
    (w1,w2,w3,errors,outputs,B1,B2)=initParams(x,y,n_hidden,max_iter)
    for i = 1:max_iter    
        (a1,h1,a2,h2,ay,yhat,err) = ffward(x,y,w1,w2,w3)
        δa2 = (B2.*err) .* dlogis(a2) 
        δa1 = (B1.*δa2) .* dlogis(a1) 
        updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err)
        errors[i] = sum(abs(err))
        outputs[:,i] = yhat
    end
    return (outputs,errors)
end

@everywhere function dfa(x,y,n_hidden,max_iter)
    (w1,w2,w3,errors,outputs,B1,B2)=initParams(x,y,n_hidden,max_iter)
    for i = 1:max_iter    
        (a1,h1,a2,h2,ay,yhat,err) = ffward(x,y,w1,w2,w3)
        δa2 = (B2.*err) .* dlogis(a2) 
        δa1 = (B1.*err) .* dlogis(a1) 
        updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err)
        errors[i] = sum(abs(err))
        outputs[:,i] = yhat
    end
    return (outputs,errors)
end




@everywhere function ifa(x,y,n_hidden,max_iter)
    (w1,w2,w3,errors,outputs,B1,_)=initParams(x,y,n_hidden,max_iter)
    for i = 1:max_iter    
        (a1,h1,a2,h2,ay,yhat,err) = ffward(x,y,w1,w2,w3)
        δa1 = (B1.*err) .* dlogis(a1) 
        δa2 = (w2 * δa1) .* dlogis(a2) 
        updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err)
        errors[i] = sum(abs(err))
        outputs[:,i] = yhat
    end
    return (outputs,errors)
end



In [7]:
x = [1 1 ; 0 1 ; 1 0 ; 0 0]'
y = [0 1 1 0]
n_hidden = 30; # Number of hidden units
max_iter = 1000; #Number of learning steps
trials = 30;

dfa_err=@parallel (hcat) for _=1:trials
    (_,dfa_errors) = dfa(x,y,n_hidden,max_iter)
    dfa_errors
end
(dfa_yhat,_) = dfa(x,y,n_hidden,max_iter)

bp_err = @parallel (hcat) for _=1:trials
    (_,bp_errors) = bp(x,y,n_hidden,max_iter)
    bp_errors
end
(bp_yhat,_) = bp(x,y,n_hidden,max_iter)

ifa_err=@parallel (hcat) for _=1:trials
    (_,ifa_errors) = ifa(x,y,n_hidden,max_iter)
    ifa_errors
end
(ifa_yhat,_) = ifa(x,y,n_hidden,max_iter);

fa_err=@parallel (hcat) for _=1:trials
    (_,fa_errors) = fa(x,y,n_hidden,max_iter)
    fa_errors
end
(fa_yhat,_) = fa(x,y,n_hidden,max_iter);

In [9]:
using Plots
l = @layout [
    a b
    c d
    e f
]
gr()
#init_notebook(true)
p1=plot(dfa_yhat',label=["1,1" "0,1" "1,0" "0,0"],title="DFA")
p2=plot(ifa_yhat',label=["1,1" "0,1" "1,0" "0,0"],title="IFA")
p3=plot(fa_yhat',label=["1,1" "0,1" "1,0" "0,0"],title="FA")
p4=plot(bp_yhat',label=["1,1" "0,1" "1,0" "0,0"],title="BP")
p5=plot([dfa_err[:,1] ifa_err[:,1] fa_err[:,1] bp_err[:,1]],label=["DFA" "IFA" "FA" "BP"],title="First Trial Error")
p6=plot([mean(dfa_err,2) mean(ifa_err,2) mean(fa_err,2) mean(bp_err,2)],label=["DFA" "IFA" "FA" "BP"],title="Mean Trials Error")
plot(p1,p2,p3,p4,p5,p6,size=(900,900),layout=l)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 250 
 
 
 500 
 
 
 750 
 
 
 1000 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 DFA 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:1.8; stroke-opacity:1; fill:none" points="
 39.4926,284.952 39.8995,284.385 40.3065,141.746 40.7135,160.219 41.1205,120.119 41.5275,207.401 41.9344,51.8057 42.3414,272.647 42.7484,32.9724 43.1554,274.201 
 43.5623,33.4224 43.9693,274.473 44.3763,33.2598 44.7833,274.52 45.1902,33.0454 45.5972,274.478 46.0042,32.8627 46.4112,274.391 46.8182,32.7256 47.2251,274.291 
 47.6321,32.6351 48.0391,274.194 48.4461,32.5883 48.853,274.109 49.26,32.5802 49.667,274.039 50.074,32.6056 50.481,273.984 50.8879,32.6598 51.2949,273.942 
 51.7019,32.7385 52.1089,273.913 52.5158,32.8385 52.9228,273.896 53.3298,32.9572 53.7368,273.888 54.1438,33.0925 54.5507,273.892 54.9577,33.2428 55.3647,273.905 
 55.7717,33.407 56.1786,273.928 56.5856,33.584 56.9926,273.962 57.3996,33.773 57.8065,274.006 58.2135,33.9734 58.6205,274.059 59.0275,34.1848 59.4345,274.123 
 59.8414,34.4066 60.2484,274.197 60.6554,34.6386 61.0624,274.28 61.4693,34.8804 61.8763,274.372 62.2833,35.1318 62.6903,274.472 63.0973,35.3925 63.5042,274.579 
 63.9112,35.6623 64.3182,274.693 64.7252,35.9412 65.1321,274.813 65.5391,36.2289 65.9461,274.936 66.3531,36.5255 66.76,275.064 67.167,36.8309 67.574,275.194 
 67.981,37.1451 68.388,275.325 68.7949,37.4682 69.2019,275.456 69.6089,37.8004 70.0159,275.588 70.4228,38.1418 70.8298,275.718 71.2368,38.4928 71.6438,275.846 
 72.0508,38.8539 72.4577,275.971 72.8647,39.2256 73.2717,276.093 73.6787,39.6086 74.0856,276.21 74.4926,40.0038 74.8996,276.324 75.3066,40.4123 75.7136,276.432 
 76.1205,40.8356 76.5275,276.535 76.9345,41.2754 77.3415,276.632 77.7484,41.7337 78.1554,276.723 78.5624,42.213 78.9694,276.808 79.3763,42.7165 79.7833,276.886 
 80.1903,43.2477 80.5973,276.957 81.0043,43.8111 81.4112,277.022 81.8182,44.4119 82.2252,277.079 82.6322,45.0564 83.0391,277.128 83.4461,45.7523 83.8531,277.169 
 84.2601,46.5086 84.6671,277.202 85.074,47.3365 85.481,277.225 85.888,48.2495 86.295,277.238 86.7019,49.2642 87.1089,277.24 87.5159,50.4013 87.9229,277.229 
 88.3299,51.6865 88.7368,277.203 89.1438,53.1523 89.5508,277.159 89.9578,54.8404 90.3647,277.094 90.7717,56.8039 91.1787,277.003 91.5857,59.1123 91.9926,276.877 
 92.3996,61.8562 92.8066,276.708 93.2136,65.1558 93.6206,276.48 94.0275,69.1737 94.4345,276.171 94.8415,74.1337 95.2485,275.745 95.6554,80.3522 96.0624,275.149 
 96.4694,88.2855 96.8764,274.291 97.2834,98.5925 97.6903,273.017 98.0973,112.178 98.5043,271.06 98.9113,130.052 99.3182,267.976 99.7252,152.55 100.132,263.199 
 100.539,177.569 100.946,256.553 101.353,200.017 101.76,249.182 102.167,215.859 102.574,243.071 102.981,225.333 103.388,239.232 103.795,230.656 104.202,237.4 
 104.609,233.68 105.016,236.878 105.423,235.519 105.83,237.075 106.237,236.773 106.644,237.617 107.051,237.75 107.458,238.302 107.865,238.598 108.272,239.034 
 108.679,239.382 109.086,239.772 109.493,240.131 109.9,240.5 110.307,240.856 110.714,241.212 111.121,241.561 111.528,241.906 111.935,242.247 112.342,242.583 
 112.748,242.915 113.155,243.243 113.562,243.567 113.969,243.887 114.376,244.202 114.783,244.514 115.19,244.822 115.597,245.126 116.004,245.426 116.411,245.723 
 116.818,246.015 117.225,246.305 117.632,246.59 118.039,246.873 118.446,247.152 118.853,247.427 119.26,247.699 119.667,247.968 120.074,248.234 120.481,248.496 
 120.888,248.756 121.295,249.012 121.702,249.265 122.109,249.515 122.516,249.763 122.923,250.007 123.33,250.249 123.737,250.488 124.144,250.724 124.551,250.957 
 124.958,251.188 125.365,251.416 125.772,251.642 126.179,251.865 126.586,252.085 126.993,252.303 127.4,252.519 127.807,252.732 128.214,252.943 128.621,253.151 
 129.028,253.357 129.435,253.561 129.842,253.763 130.249,253.963 130.656,254.16 131.062,254.355 131.469,254.548 131.876,254.74 132.283,254.929 132.69,255.116 
 133.097,255.30

# [MNIST with BP vs DFA](./mnist.ipynb)

In [60]:
mean(fa_err,2)

500×1 Array{Float64,2}:
 1.97244 
 2.01976 
 1.96826 
 2.01415 
 1.97317 
 2.0098  
 1.97192 
 2.00395 
 1.96786 
 2.00018 
 1.96239 
 1.99694 
 1.95628 
 ⋮       
 0.464423
 0.46257 
 0.459388
 0.456857
 0.452786
 0.449498
 0.444789
 0.441679
 0.437848
 0.436335
 0.433968
 0.433438